In [1]:
import os 
import sys
import re
import glob
import zipfile
from os.path import basename
from lxml import etree

In [2]:
os.chdir("..")

In [3]:
os.getcwd()

'/home/sld0465/projects/ppmi'

In [22]:
data_dir = "PPMI"
output_dir = "output"
metadata_dir = data_dir + "/metadata"

In [5]:
metadata_files = os.listdir(metadata_dir)

In [6]:
p = re.compile(r"(^PPMI)_([0-9]*)_(.*)_(S[0-9]*)_(I[0-9]*).xml")

In [7]:
metadata_splits = [re.search(p,s).groups() for s in metadata_files]

In [8]:
mfile = metadata_files[0]
mdict = dict(zip(["dir", "id", "descr", "series", "image"], metadata_splits[0]))
mfile_fullpath = metadata_dir + "/" + mfile

In [9]:
mdict

{'descr': 'Axial_PD-T2_TSE_FS',
 'dir': 'PPMI',
 'id': '3105',
 'image': 'I372313',
 'series': 'S189363'}

In [10]:
def create_dir_from_mdict(m):
    return(m["dir"] + "/" + m["id"] + "/" + m["descr"])

In [11]:
mdir = create_dir_from_mdict(mdict)
mdir

'PPMI/3105/Axial_PD-T2_TSE_FS'

In [12]:
# We do not know the exact directory location as we do not know what time and date
# this folder is named with. So we use the pattern /*/ to match it
match_str = mdir + "/*/%(series)s/%(dir)s_%(id)s_*.dcm" % mdict

In [13]:
match_str

'PPMI/3105/Axial_PD-T2_TSE_FS/*/S189363/PPMI_3105_*.dcm'

In [14]:
matched_dcms = glob.glob(match_str)

In [15]:
metaroot = etree.parse(mfile_fullpath)

In [16]:
visit_identifier = metaroot.findtext('.//visitIdentifier')
visit_identifier

'Month 24'

In [17]:
date_acquired = metaroot.findtext(".//dateAcquired")
date_acquired

'2013-04-18'

In [18]:
basename(matched_dcms[0])

'PPMI_3105_MR_Axial_PD-T2_TSE_FS_br_raw_20130515091800455_43_S189363_I372313.dcm'

In [19]:
xarfile = zipfile.ZipFile("tmp/testxar.zip", mode="w", compression=zipfile.ZIP_DEFLATED)

for filename in matched_dcms:
    xarfile.write(filename, basename(filename))

xarfile.close()